In [1]:
import os
import sys

def set_root_path():
    if os.getcwd().endswith('figures'): 
        os.chdir('../')
set_root_path()
sys.path.append('python/')
sys.path = [p for p in sys.path if "unbiased-inverse-volume-rendering" not in p]
print(os.getcwd())
print(sys.executable)
os.environ.pop("PYTHONPATH")

c:\Users\mayuf\source\repos\practical-ref\practical-inverse-rendering-of-textured-and-translucent-appearance
c:\Users\mayuf\anaconda3\envs\practical-inverse-rendering\python.exe


'C:\\Users\\mayuf\\source\\repos\\unbiased-inverse-volume-rendering\\mitsuba3\\build\\Release\\python'

In [2]:
from pathlib import Path
import mitsuba as mi

mi.set_variant('cuda_ad_rgb')

from practical_reconstruction import optimization_cli
from core import integrators
from core import bsdfs
from core import textures

integrators.register()
bsdfs.register()
textures.register()

In [3]:
def _update_light_position(params, camera_idx):
	light_positions = [
		[-2.944557817004439, 48.8898011620204, -41.4402460343309],
		[-2.944557817004439, 48.8898011620204, -41.4402460343309],
		[-2.944557817004439, 48.8898011620204, -41.4402460343309],
		[27.055610795691802, 48.81986339495665, -31.610352043829998],
		[41.3180656055926, 48.73481598693242, -3.4462441460169386],
		[4.4671667393215575, 47.1369277980911, 42.78219333472567],
		[-42.91042711255302, 47.2987396659678, 3.9648433877314604],
		[-33.3138670015183, 47.353262970810604, -27.76333185326814],
		[-4.0928649644330495, 47.3407616054259, -43.412834290840195],
		[27.63531231151012, 47.2685587001079, -33.816397641445796],
		[43.2847288874415, 47.178949737549196, -4.59548434384601],
		[33.6881687764068, 47.1244264327064, 27.132690897152997],
		[33.6881687764068, 47.1244264327064, 27.132690897152997],
		[43.2847288874415, 47.178949737549196, -4.59548434384601],
		[27.63531231151012, 47.2685587001079, -33.816397641445796],
		[-4.0928649644330495, 47.3407616054259, -43.412834290840195],
		[-33.3138670015183, 47.353262970810604, -27.76333185326814],
		[-42.91042711255302, 47.2987396659678, 3.9648433877314604],
		[4.4671667393215575, 47.1369277980911, 42.78219333472567],
		[43.2160052121517, 47.0472861267224, -4.74011971286851],
		[4.61129874588445, 47.0047545442678, 42.71339045984761],
		[-27.110711342651502, 47.076678220198616, 33.2390870886382],
		[-42.842128441427, 47.1662190544112, 4.1088962216802996],
		[-33.3677017693281, 47.2209252406101, -27.613111405679],
		[-4.237421975160156, 47.20875063686579, -43.34461395103581],
		[27.4845881133758, 47.136826960935004, -33.870310579826004],
		[27.4845881133758, 47.136826960935004, -33.870310579826004],
		[-4.237421975160156, 47.20875063686579, -43.34461395103581],
		[-33.3677017693281, 47.2209252406101, -27.613111405679],
		[-42.842128441427, 47.1662190544112, 4.1088962216802996],
		[-27.110711342651502, 47.076678220198616, 33.2390870886382],
		[4.61129874588445, 47.0047545442678, 42.71339045984761],
		[43.2160052121517, 47.0472861267224, -4.74011971286851],
		[54.346621438817294, -0.786676450502069, 24.2689468400264],
		[-24.57995504968848, -0.724855391664175, 53.815623248847],
		[-55.69939927018449, -0.601267884668769, 20.471749327836],
		[-54.1263850774641, -0.503287498684284, -25.11075868475832],
		[-20.78236285181735, -0.48830981497388004, -56.230285802138],
		[24.80019141104173, -0.565108557522178, -54.657435093579295],
		[55.9196356315378, -0.688696064517584, -21.31356117256785],
		[55.9196356315378, -0.688696064517584, -21.31356117256785],
		[24.80019141104173, -0.565108557522178, -54.657435093579295],
		[-20.78236285181735, -0.48830981497388004, -56.230285802138],
		[-54.1263850774641, -0.503287498684284, -25.11075868475832],
		[-55.69939927018449, -0.601267884668769, 20.471749327836],
		[-24.57995504968848, -0.724855391664175, 53.815623248847],
		[54.346621438817294, -0.786676450502069, 24.2689468400264],
		[54.6571493603019, 4.41870496840774, 24.41354929025032],
		[39.6303138216189, 47.0780994649407, 17.415384223131202],
		[1.6598691824590588, 65.52149640366119, -0.2676721498317754],
		[-37.0116130568947, 48.94500399409722, -18.2771252295649],
	]
	params['emit-Point.position'] = light_positions[camera_idx]
	params.update()

def save_images(scene, params, dir, spp):
    print("start rendering")
    for i, sensor in enumerate(scene.sensors()):
					_update_light_position(params, i)
											
					integrator = mi.load_dict({
							'type': 'prb_path_volume',
							'max_sss_depth': 256,
							'max_path_depth': 10,
					})
					image = mi.render(scene, sensor=sensor, integrator=integrator, spp=spp)
					bitmap = mi.Bitmap(image).convert(
									pixel_format=mi.Bitmap.PixelFormat.RGB,
									component_format=mi.Struct.Type.UInt8,
									srgb_gamma=True
					)
					png_path = dir / f"ref_view_{i:03d}.png"
					bitmap.write(str(png_path))

					bitmap = mi.Bitmap(image).convert(
													pixel_format=mi.Bitmap.PixelFormat.RGB,
													component_format=mi.Struct.Type.Float32,
													srgb_gamma=False
					)
					exr_path = dir / f"ref_view_{i:03d}.exr"
					bitmap.write(str(exr_path))
					print(f"saving: {i}")

In [ ]:
ref_spp=4096
scene_name = 'kiwi'
ref_dir=Path("third_party/kiwi/references")
ref_scene_path=f'third_party/{scene_name}/mts_scene/kiwi_ref.xml'
ref_scene = mi.load_file(ref_scene_path)
ref_params = mi.traverse(ref_scene)

save_images(ref_scene, ref_params, ref_dir, ref_spp)

In [5]:
def format_float(f):
  """Formats a float such that 0.1 becomes "0_1", 10.0 becomes "10_0", etc."""
  return str(f).replace('.', '_')

scene_name = 'kiwi'
technique = 'mipmap_pyramid'
base_learning_rate = 0.0005

skip_existing = False

print(
    f'******** Running {technique} with base learning rate'
    f' {base_learning_rate} ********'
)

override_bindings = []
result_folder = f'results/{scene_name}/{technique}'

result_folder += f'_lr_{format_float(base_learning_rate)}'
override_bindings.append(
    f'SceneConfig.base_learning_rate={base_learning_rate}'
)

# Ensure that the default tmp folder is used
override_bindings.append("SceneConfig.tmp_folder=''")
override_bindings.append(
    f'SceneConfig.use_gradient_filtering=False'
)

override_bindings.append(
    f"SceneConfig.result_folder='{result_folder}'"
)

if technique == 'gradient_filtering':
    gin_config_name = f'{scene_name}/naive'
else:
    gin_config_name = f'{scene_name}/{technique}'

print(f'Next result location: {result_folder}')
if skip_existing and Path(result_folder).exists():
    print('Skipping, already present')
else:
    # Run the config
    optimization_cli.run_config(gin_config_name, override_bindings, sss_config=True)

******** Running mipmap_pyramid with base learning rate 0.0005 ********
Next result location: results/kiwi/mipmap_pyramid_lr_0_0005
Preparing Mitsuba scene for optimization
Preparing references and sensors for optimization
Rendering camera camera_000
Reference found at: third_party\kiwi\references\ref_view_000.exr, copying to tmp folder
Rendering camera camera_001
Reference found at: third_party\kiwi\references\ref_view_001.exr, copying to tmp folder
Rendering camera camera_002
Reference found at: third_party\kiwi\references\ref_view_002.exr, copying to tmp folder
Rendering camera camera_003
Reference found at: third_party\kiwi\references\ref_view_003.exr, copying to tmp folder
Rendering camera camera_004
Reference found at: third_party\kiwi\references\ref_view_004.exr, copying to tmp folder
Rendering camera camera_005
Reference found at: third_party\kiwi\references\ref_view_005.exr, copying to tmp folder
Rendering camera camera_006
Reference found at: third_party\kiwi\references\ref_v

Loading references for sensor camera_050: 100%|██████████| 1/1 [00:00<00:00, 500.04it/s]


Preparing optimization variables
Optimizing mat-kiwi's single_scattering_albedo from default value : [[0.5, 0.5, 0.5]]
Optimizing mat-kiwi's extinction_coefficient from default value : [[10, 10, 10]]
Optimizing mat-kiwi's hg_coefficient from default value : [[0, 0, 0]]
Learning rate for (texture) mat-kiwi (single_scattering_albedo) is  0.0005
Learning rate for (scalar) mat-kiwi (extinction_coefficient) is  0.5
Learning rate for (texture) mat-kiwi (hg_coefficient) is  0.0005
Starting optimization
Resizing initial value for mat-kiwi.nested_bsdf.single_scattering_albedo.data/mat-kiwi.nested_bsdf.single_scattering_albedo.data to shape: (2048, 2048, 3)
Skipping resizing as target shape is already reached.
Resizing initial value for mat-kiwi.nested_bsdf.hg_coefficient.data/mat-kiwi.nested_bsdf.hg_coefficient.data to shape: (512, 512, 3)
Skipping resizing as target shape is already reached.


Rendering at resolution level 0 (width=436):   3%|▎         | 128/4096 [02:34<1:20:03,  1.21s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'tmp\\kiwi\\frames\\camera_000_iter_128.exr.27992.1757925731.3776846.tmp'

# Figure starts here

In [ ]:
import matplotlib.pyplot as plt
from practical_reconstruction import figutils
from matplotlib import gridspec

def figure_grid_setup(image_shape,image_crop_shape,n_columns = 7,inner_space=0.0,outer_space=0.1,figwidth=figutils.TEXT_WIDTH):
  # Image aspect ratios
  h, w = image_shape
  h_crop, w_crop = image_crop_shape

  top_inner_rows = 1
  top_inner_cols = n_columns
  # Spacing in the inner gridspec
  top_inner_wspace = inner_space
  # same vertical spacing as horizontal spacing
  top_inner_hspace = top_inner_wspace * figutils.gridspec_aspect(
      n_rows=1, n_cols=1, w=[w]*top_inner_cols, h=[h]
  )
  top_height_ratios = [h]
  top_inner_aspect = figutils.gridspec_aspect(
      n_rows=top_inner_rows,
      n_cols=top_inner_cols,
      w=[w] * top_inner_cols,
      h=top_height_ratios,
      wspace=top_inner_wspace,
      hspace=top_inner_hspace,
  )
  # print(top_inner_aspect)

  # Spacing in the main griddpec
  outer_rows = 3
  outer_cols = 1
  outer_wspace = 0.0
  outer_hspace = outer_space
  # If width is 1, we need the sum of the inverses for the height (single column)
  # If height is 1, we need the sum for the width (single row)
  outer_aspect = figutils.gridspec_aspect(
      n_rows=outer_rows,
      n_cols=outer_cols,
      w=1,
      h=[1 / top_inner_aspect] * outer_rows,
      wspace=outer_wspace,
      hspace=outer_hspace,
  )

  fig = plt.figure(
      1, figsize=(figwidth, figwidth / outer_aspect)
  )

  outer_gs = fig.add_gridspec(
      outer_rows,
      outer_cols,
      hspace=outer_hspace,
      wspace=outer_wspace,
      height_ratios=[1 / top_inner_aspect] * outer_rows,
      width_ratios=[1]*outer_cols,
  )

  top_inner_gss = []
  for row in range(outer_rows):
    top_inner_gs = gridspec.GridSpecFromSubplotSpec(
        top_inner_rows,
        top_inner_cols,
        subplot_spec=outer_gs[row],
        wspace=top_inner_wspace,
        hspace=top_inner_wspace,
        width_ratios=[h] * top_inner_cols,
        height_ratios=top_height_ratios,
    )
    top_inner_gss.append(top_inner_gs)

  return (
      fig,
      (top_inner_gss, top_inner_rows, top_inner_cols),
  )

In [ ]:
import drjit as dr
import numpy as np
from core import mitsuba_io
from core import image_util

def l2_error(ref, img):
  return dr.mean(dr.square(ref - img)).array[0]


def l1_error(ref, img):
  return dr.mean(dr.abs(ref - img)).array[0]

sensor_indices = [20, 1, 25, 12]

lr = '0_0005'
normalmap_lr = '0_01'
scene_folder = 'results/kiwi'

ref_scene_folder = (
    'third_party/kiwi/unmasked_references'
)
ref_masked_scene_folder = (
    'third_party/kiwi/references'
)
deng_scene_folder = (
    'third_party/kiwi/deng_optimized/'
)

crop_sizes = [
    (110, 110),
    (126, 126),
    (94, 94),
    (113, 113),
]

crop_offsets = [
    (159, 92),
    (150, 75),
    (168, 96),
    (157, 86),
]

our_images = []
ref_masked_images = []
ref_images = []
deng_images = []
errors_l2_ours = []
errors_l2_deng = []

boost = np.sqrt(2)

for sensor_idx, crop_size, crop_offset in zip(sensor_indices,crop_sizes,crop_offsets):

  our_filename = f'{scene_folder}/mipmap_pyramid_lr_{lr}/frames/camera_{sensor_idx:03d}_iter_4095_spp_8192.exr'
  our_image = mitsuba_io.read_bitmap(our_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  our_images.append(image_util.tonemap(boost*our_image))
  ref_masked_filename = (
      f'{ref_masked_scene_folder}/ref_view_{sensor_idx:03d}.exr'
  )
  ref_masked_image = mitsuba_io.read_bitmap(ref_masked_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  ref_masked_images.append(image_util.tonemap(ref_masked_image))

  ref_filename = f'{ref_scene_folder}/kiwi_{sensor_idx:05d}.exr'
  ref_image = mitsuba_io.read_bitmap(ref_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  ref_images.append(
      image_util.tonemap(
          boost*image_util.resize_to_width(ref_image, ref_image.size()[0] // 2)
      )
  )
  deng_filename = f'{deng_scene_folder}/optimized_{sensor_idx:03d}_spp_256.exr'
  deng_image = mitsuba_io.read_bitmap(deng_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  deng_images.append(image_util.tonemap(boost*deng_image))

  errors_l2_ours.append(
      l2_error(
          figutils.crop_image(
              mi.TensorXf(ref_masked_image), crop_offset, crop_size
          ),
          figutils.crop_image(
              mi.TensorXf(our_image), crop_offset, crop_size
          ),
      )
  )
  errors_l2_deng.append(
      l2_error(
          figutils.crop_image(
              mi.TensorXf(ref_masked_image), crop_offset, crop_size
          ),
          figutils.crop_image(
              mi.TensorXf(deng_image), crop_offset, crop_size
          ),
      )
  )

our_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(our_images)
]
ref_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(ref_images)
]
deng_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(deng_images)
]

In [ ]:
FIGURE_DIR = "figures/pdfs"
FIGURE_NAME = "deng_comparison"

def error_format(error, scale):
  return f"{error*scale:.3f}"

(
    fig,
    (top_inner_gss, top_inner_rows, top_inner_cols),
) = figure_grid_setup(
    our_images[0].shape[:2],
    our_images[0].shape[:2],
    n_columns=len(sensor_indices),
    inner_space=0.03,
    outer_space=0.15,
    figwidth=figutils.COLUMN_WIDTH,
)


scale = 1000
scale_txt = figutils.math_label(r"\text{$\times 10^3$}")

line_width = 0.75
crop_color = 'orange'
crop_color = (55/255.0,118/255.0,171/255.0,1.0)

row_titles = [
    "Deng et al. 2022",
    "Ours",
    "Reference",
]

for row, top_inner_gs in enumerate(top_inner_gss):
  if row == 0:
    images = deng_images
    errors = errors_l2_deng
  elif row == 1:
    images = our_images
    errors = errors_l2_ours
  elif row == 2:
    images = ref_images

  for col in range(top_inner_cols):
    ax = fig.add_subplot(top_inner_gs[col])
    ax.spines[:].set_color(crop_color)
    ax.spines[:].set_linewidth(line_width)
    figutils.disable_ticks(ax)
    image = images[col]

    # alpha_channel = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8) + 255
    # black_pixels = np.all(image == 0, axis=-1)
    # alpha_channel[black_pixels] = 0
    # image_with_alpha = np.dstack((image, alpha_channel))

    ax.imshow(image, aspect="equal")
    if col == 0:
      ax.set_ylabel(row_titles[row],labelpad=1.5)

    if row == 0 or row == 1:
      label = error_format(errors[col], scale)
    elif row == 2:
      label = r"$L_2$ " + scale_txt + " error" if col == 0 else ""
    ax.set_xlabel(label, labelpad=1.5)

figutils.force_post_crop_size(fig, figutils.COLUMN_WIDTH)

In [ ]:
figutils.savefig(
    fig,
    name=Path(FIGURE_NAME),
    fig_directory=Path(FIGURE_DIR),
    dpi=300,
    pad_inches=0.005,
    bbox_inches="tight",
    compress=False,
    target_width=figutils.COLUMN_WIDTH,
    backend=None,
)